In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader

In [2]:
os.chdir("/Users/eugenekim/PycharmProjects/aslAlphabetClassification")

In [3]:
from data.customDataset import ASLDataset

In [4]:
dataset = ASLDataset(csv_file = "data/aslDataset.csv", root_dir = "data/raw_data/asl_alphabet_complete", transform = transforms.ToTensor())

In [17]:
batch_size = 5
train_set, test_set = torch.utils.data.random_split(dataset, [1200, 250])
train_loader = DataLoader(dataset=train_set, batch_size = batch_size, shuffle = True)

In [19]:
for i, j in train_loader:
    print(j)
    break

tensor([18, 17,  9,  4, 27])
